In [21]:
from azureml.core import Workspace, Dataset, Datastore
import pandas as pd
import numpy as np
import os
from datetime import datetime, date, timedelta
import sklearn
import joblib
import math
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Run
import azureml.core
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import ComputeTarget
from azureml.pipeline.steps import PythonScriptStep
from azureml.core import Experiment, Environment
from azureml.core.runconfig import RunConfiguration
from azureml.pipeline.core import Pipeline

In [2]:
ws = Workspace.from_config()

In [3]:
vm_szie = "STANDARD_DS11_V2"
compute_name = "ram"
compute_target = ws.compute_targets[compute_name]

In [18]:
#declare environment
aml_config = RunConfiguration()
aml_config.target = compute_target

USE_CURATEDENV = True
if USE_CURATEDENV:
    curated_env = Environment.get(workspace= ws,
    name = "AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")
    aml_config.environment = curated_env
else:
    aml_config.environment.python.user_managed_dependencies = False
    aml_config.environment.python.conda_dependencies = CondaDependencies.create(
        conda_packages=['pandas', 'sckit-learn'],
        pip_packages=['azureml-sdk', 'azureml-dataset-runtime[fuse, pandas]', 'seaborn'],
        pin_sdk_version=False
    )

In [32]:
#pipelines
read_data = "data_wrangling.py"
prep = "preprocessing.py"
model = "modeling.py"

py_script_read_run = PythonScriptStep(
    script_name=read_data,
    compute_target=compute_target,
    arguments=['--input-data', "diabetes.csv"],
    runconfig=aml_config,
    allow_reuse=False
)

py_script_prep_run = PythonScriptStep(
    script_name=prep,
    compute_target=compute_target,
    arguments=['--prep', "wrangled.csv"],
    runconfig=aml_config,
    allow_reuse=False
)

py_script_model_run = PythonScriptStep(
    script_name=model,
    compute_target=compute_target,
    arguments=['--train', "preprocessed.csv"],
    runconfig=aml_config,
    allow_reuse=False
)

pipeline_step = [py_script_read_run, py_script_prep_run, py_script_model_run]
pipeline1 = Pipeline(workspace=ws, steps=[pipeline_step])

In [33]:
#Experiment1
pipeline_run1 = Experiment(ws, "First_run_exp").submit(pipeline1)
pipeline_run1.wait_for_completion(show_output=True)

Created step data_wrangling.py [85cc9d17][79a01c7b-cd05-4df1-85fd-edc9e015022f], (This step will run and generate new outputs)
Created step preprocessing.py [101dc3cb][8ef39a78-2123-4c34-93e4-025681fbd617], (This step will run and generate new outputs)
Created step modeling.py [12d28cd8][9cf0b219-2895-486e-a161-38a766479628], (This step will run and generate new outputs)
Submitted PipelineRun 0f78090e-ebe3-4a3c-b4f7-fc82a68949b6
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/0f78090e-ebe3-4a3c-b4f7-fc82a68949b6?wsid=/subscriptions/82cfc0a9-b871-4f48-bb9a-0f9e00faacf9/resourcegroups/ram/workspaces/ram&tid=48dbabfc-37fa-4dd4-913c-cd7091a8ea08
PipelineRunId: 0f78090e-ebe3-4a3c-b4f7-fc82a68949b6
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/0f78090e-ebe3-4a3c-b4f7-fc82a68949b6?wsid=/subscriptions/82cfc0a9-b871-4f48-bb9a-0f9e00faacf9/resourcegroups/ram/workspaces/ram&tid=48dbabfc-37fa-4dd4-913c-cd7091a8ea08
PipelineRun Status: NotStarted
PipelineRun Stat

'Finished'

In [35]:
#Experiment2
pipeline_run2 = Experiment(ws, "Second_run_exp").submit(pipeline1)
pipeline_run2.wait_for_completion(show_output=True)

Submitted PipelineRun de7ff97b-62fe-4e71-9d75-59b0de6ec530
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/de7ff97b-62fe-4e71-9d75-59b0de6ec530?wsid=/subscriptions/82cfc0a9-b871-4f48-bb9a-0f9e00faacf9/resourcegroups/ram/workspaces/ram&tid=48dbabfc-37fa-4dd4-913c-cd7091a8ea08
PipelineRunId: de7ff97b-62fe-4e71-9d75-59b0de6ec530
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/de7ff97b-62fe-4e71-9d75-59b0de6ec530?wsid=/subscriptions/82cfc0a9-b871-4f48-bb9a-0f9e00faacf9/resourcegroups/ram/workspaces/ram&tid=48dbabfc-37fa-4dd4-913c-cd7091a8ea08
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 8d060ae9-6098-4402-80cc-46de3fcbeb4a
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/8d060ae9-6098-4402-80cc-46de3fcbeb4a?wsid=/subscriptions/82cfc0a9-b871-4f48-bb9a-0f9e00faacf9/resourcegroups/ram/workspaces/ram&tid=48dbabfc-37fa-4dd4-913c-cd7091a8ea08
StepRun( modeling.py ) Status: NotStarted
StepRun( modeling.py ) Status: Runni

'Finished'

In [34]:
pipeline_run1

Experiment,Id,Type,Status,Details Page,Docs Page
First_run_exp,0f78090e-ebe3-4a3c-b4f7-fc82a68949b6,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [36]:
datastore = Datastore.get(ws, "workspaceblobstore")
datastore.download(target_path="azureml", prefix="model_estimator", overwrite=False)

Downloaded model_estimator_500.pkl, 1 files out of an estimated total of 1


1

In [38]:
from azureml.core.model import Model

fine_tuned_model = Model.register(model_name="rf_model_500",
model_path="./azureml/model_estimator_500.pkl",
tags={},
description="Diabetes model",
workspace=ws)

Registering model rf_model_500


In [39]:
model = Model(ws, name='rf_model_500')
print('Loaded model version', model.version)

Loaded model version 2


In [40]:
from azureml.core import Environment

env = Environment("deploytocloudenv")
env.python.conda_dependencies.add_pip_package("joblib")
env.python.conda_dependencies.add_pip_package("numpy==1.23")
env.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))

In [41]:
%%writefile score3.py
import joblib
import json
import numpy as np

from azureml.core.model import Model

def init():
    global model_3
    model_3_path = Model.get_model_path(model_name='rf_model_500')
    model_3 = joblib.load(model_3_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = np.array(data)
        result_1 = model_3.predict(data)
        
        return {"prediction1": result_1.tolist()}
    except Exception as e:
        result = str(e)
        return result

Writing score3.py


In [44]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score3.py", environment=env)

from azureml.core.webservice import AciWebservice

aci_service_name = "aciservice-modelrfdiabetes"

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(ws, aci_service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)

print(service.state)

/tmp/ipykernel_5489/4223386439.py:11: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws, aci_service_name, [model], inference_config, deployment_config, overwrite=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-02-03 17:26:35+00:00 Creating Container Registry if not exists.
2023-02-03 17:26:38+00:00 Use the existing image.
2023-02-03 17:26:39+00:00 Generating deployment configuration.
2023-02-03 17:26:40+00:00 Submitting deployment to compute..
2023-02-03 17:26:48+00:00 Checking the status of deployment aciservice-modelrfdiabetes..
2023-02-03 17:28:12+00:00 Checking the status of inference endpoint aciservice-modelrfdiabetes.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [46]:
datastore = Datastore.get(ws, "workspaceblobstore")
df = Dataset.Tabular.from_delimited_files(path=[(datastore, "preprocessed.csv")]).to_pandas_dataframe()
df.head()

,Column1,Pregnancies,Glucose,SkinThickness,BMI,Age,Outcome
0,0,0.747718,0.810300,0.801825,0.591265,0.889831,1.0
1,1,0.232725,0.091265,0.644720,0.213168,0.558670,0.0
2,2,0.863755,0.956975,0.357888,0.077575,0.585398,1.0
3,3,0.232725,0.124511,0.357888,0.284224,0.000000,0.0
4,4,0.000000,0.721643,0.801825,0.926988,0.606258,1.0


In [47]:
df.columns

Index(['Column1', 'Pregnancies', 'Glucose', 'SkinThickness', 'BMI', 'Age',
       'Outcome'],
      dtype='object')

In [48]:
df = df[['Pregnancies', 'Glucose', 'SkinThickness', 'BMI', 'Age',
       'Outcome']]
df_new = np.array(df)

In [49]:
import json
test_sample = json.dumps({'data':df_new[:5].tolist()})
predictions = service.run(test_sample)
predictions

{'prediction1': [1.0, 0.0, 0.0, 0.0, 1.0]}